# Erwartungswert

In [11]:
from qib.operator import FieldOperator, FieldOperatorTerm, IFOType, IFODesc
from qib.field import Field
import fermitensor as ftn # after installing fermitensor (installed with kernel 3.10.11)

import hamiltonian_eq_9 as ham
import hamiltonian_eq_9_tests as ham_test
import matrix_reference as ref

import numpy as np
# from scipy import sparse
from scipy.sparse.linalg import norm

### Hamiltonian for TTN

In [2]:
L = 8
L_1 = L//2
A = range(L_1)
B = range(L_1, L)

# latt = qib.lattice.FullyConnectedLattice((L,))
# field = qib.field.Field(qib.field.ParticleType.FERMION, latt)
# tkin, vint = construct_random_coefficients(L)

# create MolecularHamiltonian Object
H_ref = ref.construct_random_molecular_hamiltonian(L)
# H_ref.vint = np.zeros_like(H_ref.vint)
field = H_ref.field
tkin = H_ref.tkin
vint = H_ref.vint


#### Create all components in (11)-(13)

In [3]:
# final Hamiltonian according to equ. (9)
H_A = ham.get_part_of_H_as_FO(A, field, tkin, vint)
H_B = ham.get_part_of_H_as_FO(B, field, tkin, vint)

H_AB = ham.get_H_AB_as_FO(A, B, field, tkin, vint)

In [4]:
H = H_A + H_B + H_AB
# H = H_A + H_B

In [5]:
ham_test.is_hermitian(H)

True

In [6]:
# compare new hamiltonian with other
# norm der differenz sollte 0 sein
norm(H.as_matrix()- H_ref.as_matrix())

35.19293891196457

In [7]:
# assert ham_test.csr_allclose(H.as_matrix(), H_ref.as_matrix(), atol=1e-5)

In [16]:
def construct_complementary_p(i: int, j: int, sites, field: Field, vint):
    """
    Construct the complementary operator `P_{ij}^B` in Eq. (11).
    """
    L = field.lattice.nsites
    vint = np.asarray(vint)
    assert vint.shape == (L, L, L, L)
    # indicator function
    eb = np.zeros(L)
    for k in sites:
        eb[k] = 1
    return FieldOperator([
        FieldOperatorTerm(
            [IFODesc(field, IFOType.FERMI_ANNIHIL),
             IFODesc(field, IFOType.FERMI_ANNIHIL)],
            [[0.5*vint[i, j, l, k]*eb[l]*eb[k] for l in range(L)] for k in range(L)])])

def construct_complementary_q(i: int, j: int, sites, field: Field, vint):
    """
    Construct the complementary operator `Q_{ij}^B` in Eq. (12).
    """
    L = field.lattice.nsites
    vint = np.asarray(vint)
    assert vint.shape == (L, L, L, L)
    # indicator function
    eb = np.zeros(L)
    for k in sites:
        eb[k] = 1
    return FieldOperator([
        FieldOperatorTerm(
            [IFODesc(field, IFOType.FERMI_CREATE),
             IFODesc(field, IFOType.FERMI_ANNIHIL)],
            [[0.5*((vint[i, k, j, l] + vint[k, i, l, j])/2 - vint[i, k, l, j])*eb[l]*eb[k] for l in range(L)] for k in range(L)])])

def construct_complementary_s(i: int, sites, field: Field, tkin, vint):
    """
    Construct the complementary operator `S_i^B` in Eq. (13).
    """
    L = field.lattice.nsites
    tkin = np.asarray(tkin)
    vint = np.asarray(vint)
    assert tkin.shape == (L, L)
    assert vint.shape == (L, L, L, L)
    # indicator function
    eb = np.zeros(L)
    for k in sites:
        eb[k] = 1
    skin = FieldOperatorTerm([IFODesc(field, IFOType.FERMI_ANNIHIL)], [0.5*tkin[i, j]*eb[j] for j in range(L)])
    sint = FieldOperatorTerm([IFODesc(field, IFOType.FERMI_CREATE),
                                           IFODesc(field, IFOType.FERMI_ANNIHIL),
                                           IFODesc(field, IFOType.FERMI_ANNIHIL)],
                                          [[[0.5*(vint[i, j, l, k] - vint[j, i, l, k])*eb[j]*eb[l]*eb[k] for l in range(L)] for k in range(L)] for j in range(L)])
    return FieldOperator([skin, sint])


In [15]:
P = ham.get_P_as_FO(A, 0, 1, field, vint)
P_ref = construct_complementary_p(0, 1, A, field, vint)
norm(P.as_matrix() - P_ref.as_matrix())

3.028524849753644

In [ ]:
Q = ham.get_Q_as_FO(A, 0, 1, field, vint)
Q_ref = construct_complementary_q(0, 1, A, field, vint)
norm(Q.as_matrix() - Q_ref.as_matrix())

In [ ]:
S = ham.get_S_as_FO(A, 0, field, tkin, vint)
S_ref = construct_complementary_s(0, A, field, tkin, vint)
norm(S.as_matrix() - S_ref.as_matrix())

In [ ]:
"""
    Hamiltonian construction acc. to eq. (9)
"""

from qib.operator import FieldOperator, FieldOperatorTerm, IFOType, IFODesc
import numpy as np

def get_part_of_H_as_FO(field, tkin, vint, subsystem):
    coeffs = np.copy(tkin)
    # complementary indices
    compl = [i for i in range(field.lattice.nsites) if i not in subsystem]
    coeffs[compl, :] = 0
    coeffs[:, compl] = 0
    # kinetic hopping term
    T = FieldOperatorTerm([IFODesc(field, IFOType.FERMI_CREATE),
                            IFODesc(field, IFOType.FERMI_ANNIHIL)],
                            coeffs)
    
    coeffs = np.copy(vint)
    coeffs[compl, :, :, :] = 0
    coeffs[:, compl, :, :] = 0
    coeffs[:, :, compl, :] = 0
    coeffs[:, :, :, compl] = 0
    # interaction term
    V = FieldOperatorTerm([IFODesc(field, IFOType.FERMI_CREATE),
                            IFODesc(field, IFOType.FERMI_CREATE),
                            IFODesc(field, IFOType.FERMI_ANNIHIL),
                            IFODesc(field, IFOType.FERMI_ANNIHIL)],
                            0.5 * coeffs.transpose((0, 1, 3, 2)))
    return FieldOperator([T, V])

def get_P_as_FO(field, vint, subsystem, i, j):
    coeffs = np.copy(vint[i, j, :, :])
    # complementary indices
    compl = [i for i in range(field.lattice.nsites) if i not in subsystem]
    coeffs[compl, :] = 0
    coeffs[:, compl] = 0
    V = FieldOperatorTerm([IFODesc(field, IFOType.FERMI_ANNIHIL),
                            IFODesc(field, IFOType.FERMI_ANNIHIL)],
                            coeffs.transpose((1, 0)))
    return FieldOperator([V])
    
def get_Q_as_FO(field, vint, subsystem, i, j):
    coeffs = vint[i, :, j, :] - vint[i, :, :, j]
    # complementary indices
    compl = [i for i in range(field.lattice.nsites) if i not in subsystem]
    coeffs[compl, :] = 0
    coeffs[:, compl] = 0
    V = FieldOperatorTerm([IFODesc(field, IFOType.FERMI_CREATE),
                            IFODesc(field, IFOType.FERMI_ANNIHIL)],
                            coeffs)
    return FieldOperator([V])

def get_S_as_FO(field, tkin, vint, subsystem, i):
    coeffs = np.copy(tkin[i, :])
    # complementary indices
    compl = [i for i in range(field.lattice.nsites) if i not in subsystem]
    coeffs[compl] = 0
    # kinetic hopping term
    T = FieldOperatorTerm([IFODesc(field, IFOType.FERMI_ANNIHIL)],
                            coeffs)
    
    coeffs = np.copy(vint[i, :, :, :])
    coeffs[compl, :, :] = 0
    coeffs[:, compl, :] = 0
    coeffs[:, :, compl] = 0
    # interaction term
    V = FieldOperatorTerm([IFODesc(field, IFOType.FERMI_CREATE),
                            IFODesc(field, IFOType.FERMI_ANNIHIL),
                            IFODesc(field, IFOType.FERMI_ANNIHIL)],
                            coeffs.transpose((0, 2, 1)))
    return FieldOperator([T, V])

def single_term_FO(field, otype, i):
    # single term Field Operator acting on only site i
    coeffs = np.zeros(field.lattice.nsites)
    coeffs[i] = 1
    return FieldOperator([FieldOperatorTerm([IFODesc(field, otype)], coeffs)])

def adjoint(P: FieldOperator):
    # get adjoint of Field Operator
    ret = []

    for term in P.terms:
        opdesc = []
        for desc in term.opdesc[::-1]: # reverse order
            opdesc += [IFODesc(desc.field, IFOType.adjoint(desc.otype))] # replace create with annihil and vice versa
        coeffs = np.copy(term.coeffs).conjugate().T # reverse order
        
        ret.append(FieldOperatorTerm(opdesc, coeffs)) # create adjoint term

    return FieldOperator(ret)

def get_H_AB_as_FO(A, B, field, tkin, vint):
    # acc. to equ. (10) in paper
    
    S_i = sum((single_term_FO(i, IFOType.FERMI_CREATE, field) 
               @ get_S_as_FO(B, i, field, tkin, vint) 
               for i in A), FieldOperator([]))
    S_j = sum((single_term_FO(j, IFOType.FERMI_CREATE, field) 
               @ get_S_as_FO(A, j, field, tkin, vint)  
               for j in B), FieldOperator([]))
    Q_ii = sum((single_term_FO(i, IFOType.FERMI_CREATE, field) 
                @ single_term_FO(i, IFOType.FERMI_ANNIHIL, field) 
                @ get_Q_as_FO(B, i, i, field, vint) 
                for i in A), FieldOperator([]))
     
    # i > j in A
    P_ij = FieldOperator([])
    Q_ij = FieldOperator([])
    for i in A:
        for j in A:
            P_ij += single_term_FO(i, IFOType.FERMI_CREATE, field) @ single_term_FO(j, IFOType.FERMI_CREATE, field) @ get_P_as_FO(B, i, j, field, vint)
            Q_ij += single_term_FO(i, IFOType.FERMI_CREATE, field) @ single_term_FO(j, IFOType.FERMI_ANNIHIL, field) @ get_Q_as_FO(B, i, j, field, vint)

    
    P_ij = sum(single_term_FO(i, IFOType.FERMI_CREATE, field) 
                @ single_term_FO(j, IFOType.FERMI_CREATE, field) 
                @ get_P_as_FO(B, i, j, field, vint) 
                for i in A for j in A)
    
    Q_ij = sum(single_term_FO(i, IFOType.FERMI_CREATE, field) 
                @ single_term_FO(j, IFOType.FERMI_ANNIHIL, field) 
                @ get_Q_as_FO(B, i, j, field, vint) 
                for i in A for j in A)

    firstLine = S_i + S_j #+ Q_ii
    secLine = P_ij + Q_ij
    return firstLine + secLine + adjoint(firstLine) + adjoint(secLine) 


In [ ]:
def csr_allclose(a, b, rtol=1e-6, atol=1e-8):
    c = np.abs(a - b)# - rtol * np.abs(b)
    help = np.round(c,6)
    help.eliminate_zeros()
    # print(help)
    print(help.max())
    return c.max() <= atol

def reference_S(field, tkin, vint, first_sub, sec_sub):
    coeffs = np.copy(tkin)
    # complementary indices
    compl_first = [i for i in range(field.lattice.nsites) if i not in first_sub]
    compl_sec = [i for i in range(field.lattice.nsites) if i not in sec_sub]
    coeffs[compl_first, :] = 0
    coeffs[:, compl_sec] = 0
    # kinetic hopping term
    T = FieldOperatorTerm([IFODesc(field, IFOType.FERMI_CREATE),
                            IFODesc(field, IFOType.FERMI_ANNIHIL)],
                            coeffs)
    
    coeffs = np.copy(vint)
    coeffs[compl_first, :, :, :] = 0
    coeffs[:, compl_sec, :, :] = 0
    coeffs[:, :, compl_sec, :] = 0
    coeffs[:, :, :, compl_sec] = 0
    # interaction term
    V = FieldOperatorTerm([IFODesc(field, IFOType.FERMI_CREATE),
                            IFODesc(field, IFOType.FERMI_CREATE),
                            IFODesc(field, IFOType.FERMI_ANNIHIL),
                            IFODesc(field, IFOType.FERMI_ANNIHIL)],
                            coeffs.transpose((0, 1, 3, 2)))

    return FieldOperator([T, V])
    
# TODO
def reference_Q(field, vint, A, B):
    coeffs = np.copy(vint)
    # complementary indices
    compl_A = [i for i in range(field.lattice.nsites) if i not in A]
    compl_B = [i for i in range(field.lattice.nsites) if i not in B]
    coeffs[compl_A, :, :, :] = 0
    coeffs[:, compl_B, :, :] = 0
    coeffs[:, :, compl_B, :] = 0
    coeffs[:, :, :, compl_B] = 0
    # interaction term
    V = FieldOperatorTerm([IFODesc(field, IFOType.FERMI_CREATE),
                            IFODesc(field, IFOType.FERMI_CREATE),
                            IFODesc(field, IFOType.FERMI_ANNIHIL),
                            IFODesc(field, IFOType.FERMI_ANNIHIL)],
                            coeffs.transpose((0, 1, 3, 2)))

    return FieldOperator([V])

# TODO
def reference_P(field, vint, A, B):
    coeffs = np.copy(vint)
    # complementary indices
    compl_A = [i for i in range(field.lattice.nsites) if i not in A]
    compl_B = [i for i in range(field.lattice.nsites) if i not in B]
    coeffs[compl_A, :, :, :] = 0
    coeffs[:, compl_B, :, :] = 0
    coeffs[:, :, compl_B, :] = 0
    coeffs[:, :, :, compl_B] = 0
    # interaction term
    V = FieldOperatorTerm([IFODesc(field, IFOType.FERMI_CREATE),
                            IFODesc(field, IFOType.FERMI_CREATE),
                            IFODesc(field, IFOType.FERMI_ANNIHIL),
                            IFODesc(field, IFOType.FERMI_ANNIHIL)],
                            coeffs.transpose((0, 1, 3, 2)))

    return FieldOperator([V])


### Test components

In [ ]:
S_i = sum((ham.single_term_FO(i, IFOType.FERMI_CREATE, field) 
            @ ham.get_S_as_FO(B, i, field, tkin, vint) 
            for i in A), FieldOperator([]))

S_ref = ham_test.reference_S(field, tkin, vint, A, B)

In [ ]:
norm(S_i.as_matrix()- S_ref.as_matrix())

1.350252414126402e-15

In [ ]:
ham_test.is_hermitian(S_ref)

False

In [ ]:
S_j = sum((ham.single_term_FO(j, IFOType.FERMI_CREATE, field) 
            @ ham.get_S_as_FO(A, j, field, tkin, vint)  
            for j in B), FieldOperator([]))

S_ref = ham_test.reference_S(field, tkin, vint, B, A)

In [ ]:
norm(S_j.as_matrix()- S_ref.as_matrix())

2.067650669371107e-15

# TODO

In [ ]:
Q_ii = sum((ham.single_term_FO(i, IFOType.FERMI_CREATE, field) 
            @ ham.single_term_FO(i, IFOType.FERMI_ANNIHIL, field) 
            @ ham.get_Q_as_FO(B, i, i, field, vint) 
            for i in A), FieldOperator([]))

Q_ref = ham_test.reference_Q(field, vint, B, A)

In [ ]:
norm(Q_ii.as_matrix()- Q_ref.as_matrix())

22.534691371888645

In [ ]:
P_ij = FieldOperator([])
for i in A:
    for j in range(i):
        P_ij += ham.single_term_FO(i, IFOType.FERMI_ANNIHIL, field) @ ham.single_term_FO(j, IFOType.FERMI_ANNIHIL, field) @ ham.adjoint(ham.get_P_as_FO(B, i, j, field, vint))
        
P_ref = ham_test.reference_P(field, vint, B, A)

In [ ]:
norm(P_ij.as_matrix()- P_ref.as_matrix())

19.75303051302453

In [ ]:
Q_ij = FieldOperator([])
for i in A:
    for j in range(i):
        Q_ij += ham.single_term_FO(i, IFOType.FERMI_CREATE, field) @ ham.single_term_FO(j, IFOType.FERMI_ANNIHIL, field) @ ham.get_Q_as_FO(B, i, j, field, vint)

Q_ref = ham_test.reference_Q(field, vint, B, A)

In [ ]:
norm(Q_ij.as_matrix()- Q_ref.as_matrix())

24.959016768606578